In [ ]:
# !pip install -U sentence-transformers

In [ ]:
# from sentence_transformers import SentenceTransformer
# import pandas as pd


# df = pd.read_csv("../data/dataCardsclean.csv")

# # Carga el modelo preentrenado de embeddings
# model = SentenceTransformer("all-MiniLM-L6-v2")  # Ligero y rápido, 384 dimensiones

# # Genera los embeddings del texto
# embeddings = model.encode(df["oracle_text"].tolist())

# # Convierte a DataFrame para visualizar mejor
# df_embeddings = pd.DataFrame(embeddings, columns=[f"embed_{i}" for i in range(embeddings.shape[1])])

# # Une los embeddings al dataframe original
# df_final = pd.concat([df, df_embeddings], axis=1)

# # Ver un ejemplo
# print(df_final.head())


In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from tqdm import tqdm  # Para ver el progreso

c:\Users\Edurne\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

df = pd.read_csv("../data/dataCardsclean.csv")

model = SentenceTransformer("all-MiniLM-L6-v2")

# Texto
texts = df["oracle_text"].tolist()

# Parámetros
batch_size = 1000
embeddings = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    batch_embeddings = model.encode(batch)
    embeddings.extend(batch_embeddings)

# Convertimos a DataFrame
df_embeddings = pd.DataFrame(embeddings, columns=[f"embed_{i}" for i in range(len(embeddings[0]))])

# Unimos al original
df_final = pd.concat([df.reset_index(drop=True), df_embeddings], axis=1)


100%|██████████| 85/85 [16:53<00:00, 11.93s/it]


In [3]:
df_final[["name","edhrec_rank"]]

,name,edhrec_rank
0,Fury Sliver,19733.0
1,Kor Outfitter,10113.0
2,Siren Lookout,11762.0
3,Web,4277.0
4,Wastewood Verge,24857.0
...,...,...
84891,Aggressive Biomancy,22237.0
84892,Horned Troll,2151.0
84893,Faerie Bladecrafter,23668.0
84894,Exultant Skymarcher,6063.0


In [2]:
df = pd.read_csv("../data/dataCardsclean.csv")
df_final = pd.read_csv("../data/dataCardsEmbeded.csv")

In [7]:
df_final['edhrec_rank'] = df_final['edhrec_rank'].max() - df_final['edhrec_rank']

In [3]:
df_final["legal_vintage"] = df["legal_vintage"]

In [4]:
df_final["log_price"] = df["log_price"]

In [5]:
df_final.to_csv("../data/dataCardsEmbeded.csv", index=False)

## Aplicar PCA para reducir dimensionalidad

No sé si tiene sentido, pero bueno. Hay que aprender a pedradas.

In [4]:
from sklearn.decomposition import PCA
import pandas as pd

# 1. Seleccionar columnas de embeddings
bert_cols = [col for col in df_final.columns if col.startswith('embed_')]
X_bert = df_final[bert_cols]

# 2. Aplicar PCA conservando ~95% de la varianza
pca = PCA(n_components=0.95, random_state=42)
bert_pca = pca.fit_transform(X_bert)

# 3. Crear nuevo DataFrame con los componentes principales
bert_pca_df = pd.DataFrame(bert_pca, columns=[f'bert_pca_{i}' for i in range(bert_pca.shape[1])])

# 4. Concatenar con el df original (sin los embed_)
df_final = pd.concat([df_final.drop(columns=bert_cols).reset_index(drop=True), bert_pca_df.reset_index(drop=True)], axis=1)

# (Opcional) Ver cuántos componentes se conservaron
print(f"Componentes PCA conservados: {bert_pca.shape[1]}")

Componentes PCA conservados: 177


In [6]:
df_final.to_csv("../data/dataCardsPCAded.csv", index=False)